In [1]:
#all libraries
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn import model_selection
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from collections import Counter
from sklearn.metrics import confusion_matrix
import sklearn.datasets as datasets
from sklearn.metrics import classification_report as report
from sklearn.datasets import make_classification
from numpy import argmax
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

In [2]:
cnames=[ "Shape",'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry',  'material_id',  'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal',  'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_vrh', 'g_vrh','e_total', 'e_ionic', 'e_electronic', 'n', 'e_ij_max', 'weighted_surface_energy_EV_PER_ANG2', 'weighted_surface_energy', 'weighted_work_function', 'surface_anisotropy', 'shape_factor', 'has_reconstructed', 'possible_species',  'theoretical']


In [3]:
df1 = pd.read_csv("materials_data.csv",names=cnames)

In [4]:
df=df1[1:]


In [5]:
d1=['elements', 'composition_reduced', 'formula_anonymous', 'chemsys', 'grain_boundaries', 'weighted_surface_energy', 'weighted_work_function', 'surface_anisotropy', 'shape_factor', 'has_reconstructed', 'weighted_surface_energy_EV_PER_ANG2','k_vrh', 'g_vrh','e_total', 'e_ionic', 'e_electronic', 'n', 'e_ij_max']
d1=list(d1)
df = df.drop(d1, axis=1)
pd.set_option('display.max_columns', None)


features=['nsites', 'volume', 'density', 'density_atomic', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable','band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal',  'is_magnetic','ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites','theoretical']
   

In [6]:
unique = df['is_gap_direct'].unique()
print(unique)

['False' 'True']


In [7]:
df['Shape'] = df['Shape'].replace(['Cubic'], '0') 
df['Shape'] = df['Shape'].replace(['Orthorhombic'], '1')
df['Shape'] = df['Shape'].replace(['Rhombohedral'], '2')
df['Shape'] = df['Shape'].replace(['Tetragonal'], '3')

df.drop(df.index[(df["Shape"] == "Monoclinic")],axis=0,inplace=True)
df.drop(df.index[(df["Shape"] == "Hexagonal")],axis=0,inplace=True)
df.drop(df.index[(df["Shape"] == "Trigonal")],axis=0,inplace=True)
df.drop(df.index[(df["Shape"] == "Trigonal")],axis=0,inplace=True)
df['is_stable'] = df['is_stable'].replace(['True'], '1')
df['is_stable'] = df['is_stable'].replace(['False'], '0')
df.fillna('0', inplace=True)
df = df.replace('False','0')
df = df.replace('True', '1')
df['ordering'] = df['ordering'].replace(['Unknown'], '-1')  
df['ordering'] = df['ordering'].replace(['NM'], '1') 
df['ordering'] = df['ordering'].replace(['AFM'], '2') 
df['ordering'] = df['ordering'].replace(['FM'], '3') 
df['ordering'] = df['ordering'].replace(['FiM'], '4') 


In [8]:
df['vbm'] = df['vbm'].replace(['Unknown'], '-1')
df['cbm'] = df['cbm'].replace(['Unknown'], '-1')

In [9]:
df

,Shape,nsites,nelements,composition,formula_pretty,volume,density,density_atomic,symmetry,material_id,uncorrected_energy_per_atom,energy_per_atom,formation_energy_per_atom,energy_above_hull,is_stable,band_gap,cbm,vbm,efermi,is_gap_direct,is_metal,is_magnetic,ordering,total_magnetization,total_magnetization_normalized_vol,total_magnetization_normalized_formula_units,num_magnetic_sites,num_unique_magnetic_sites,types_of_magnetic_species,possible_species,theoretical
1,0,5,3,Al1 Pb1 O3,AlPbO3,56.3792677148114,8.311042316450266,11.27585354296228,crystal_system=<CrystalSystem.cubic: 'Cubic'> ...,mp-1183154,-6.239539886,-6.651739886,-2.1920820792500004,0.2562708705909085,0,0.0,0,0,6.45797942,0,1,0,1,0.0325802,0.0005778755439819389,0.0325802,0,0,[],[],1
2,3,20,4,Ba4 Co2 Mo2 O12,Ba2CoMoO6,277.16717209981965,6.296939520696896,13.858358604990983,crystal_system=<CrystalSystem.tet: 'Tetragonal...,mp-561894,-6.708348041000001,-7.604548041000001,-2.4573430357500015,0.00921575099998595,0,1.274,4.2504,2.9764,3.06245207,1,0,1,2,2.1e-06,7.576654854506725e-09,1.05e-06,2,1,[Element Co],"['Ba2+', 'Mo5+', 'Co3+', 'O2-']",1
3,0,20,4,Ba4 Co2 Mo2 O12,Ba2CoMoO6,276.7660678753778,6.306065383062162,13.838303393768891,crystal_system=<CrystalSystem.cubic: 'Cubic'> ...,mp-18967,-6.717563792,-7.613763792,-2.4665587867500007,0.0,1,0.0,0,0,3.72736372,0,1,1,2,0.0109821,3.9680081031266515e-05,0.00549105,2,1,[Element Co],"['O2-', 'Mo5+', 'Ba2+', 'Co3+']",0
4,6,30,5,Ba6 Ho2 Ir2 Ru2 O18,Ba3HoIrRuO9,448.9989937567705,7.50160685500972,14.966633125225684,crystal_system=<CrystalSystem.hex_: 'Hexagonal...,mp-1228398,-7.048855328,-7.4610553280000005,-2.5962307405833345,0.0,1,0.0,0,0,3.77848706,0,1,1,3,4.0005892,0.008910018186292817,2.0002946,2,1,[Element Ru],"['Ba2+', 'Ir4+', 'Ru5+', 'Ho3+', 'O2-']",1
5,6,30,4,Ba6 Li2 Os4 O18,Ba3LiOs2O9,428.63907998177865,7.30924347300705,14.287969332725956,crystal_system=<CrystalSystem.hex_: 'Hexagonal...,mp-542207,-6.930970687333333,-7.343170687333333,-2.367118624138889,0.0,1,0.0,3.4323,3.6554,3.51387473,0,1,1,3,6.0111886,0.014023893015670745,3.0055943,4,1,[Element Os],"['O2-', 'Os5.5+', 'Ba2+', 'Li+']",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14519,0,10,5,Sr1 Eu1 Sn1 Sb1 O6,SrEuSnSbO6,145.19248463122779,6.588179794292343,14.519248463122779,crystal_system=<CrystalSystem.cubic: 'Cubic'> ...,mp-1521767,-7.013989436,-7.426189436,-2.449669374416666,0.1336846149843831,0,0.0,0,0,6.84167092,0,1,1,3,6.8116817,0.04691483665494732,6.8116817,1,1,[Element Eu],"['Sn4+', 'Eu3+', 'O2-', 'Sb3+', 'Sr2+']",1
14520,1,10,4,Sr1 Eu1 Zr2 O6,SrEuZr2O6,143.87265788442272,5.978942826644109,14.387265788442273,crystal_system=<CrystalSystem.ortho: 'Orthorho...,mp-1516749,-9.082454027999999,-9.494654028,-3.6224511499166665,0.03264155900000176,0,0.0,0,0,5.15628393,0,1,1,3,6.9999055,0.0486534801186702,6.9999055,1,1,[Element Eu],"['O2-', 'Zr4+', 'Eu2+', 'Sr2+']",1
14521,0,10,5,Sr1 Nd1 Ti1 Nb1 O6,SrNdTiNbO6,129.16347550154512,6.02477887435746,12.916347550154512,crystal_system=<CrystalSystem.cubic: 'Cubic'> ...,mp-1517850,-8.291060640000001,-8.70326064,-3.2898036190000006,0.15870182249998877,0,0.0,0,0,6.57630231,0,1,0,1,0.3933132,0.0030450806504915932,0.3933132,0,0,[],"['O2-', 'Nb4+', 'Ti4+', 'Sr2+', 'Nd2+']",1
14522,0,5,3,Zn1 N1 F3,ZnNF3,63.814802922926106,3.54957837564461,12.76296058458522,crystal_system=<CrystalSystem.cubic: 'Cubic'> ...,mp-1207389,-3.359968648,-3.6371686480000003,-0.5710335605000004,0.9907668865000003,0,0.0,0,0,-2.34447783,0,1,1,3,2.4579736,0.038517295163767534,2.4579736,1,1,[Element N],"['F-', 'Zn2+', 'N+']",1


In [10]:
mask = df == 'Unknown'
row_index, col_index = np.where(mask)
print(row_index,col_index)

[] []


In [11]:
unique_NM = df['ordering'].unique()
print(unique_NM)

['1' '2' '3' '4' '-1']


cnames=[ "Shape",'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry',  'material_id',  'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal',  'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_vrh', 'g_vrh','e_total', 'e_ionic', 'e_electronic', 'n', 'e_ij_max', 'weighted_surface_energy_EV_PER_ANG2', 'weighted_surface_energy', 'weighted_work_function', 'surface_anisotropy', 'shape_factor', 'has_reconstructed', 'possible_species',  'theoretical']


In [12]:
features=['nsites', 'volume', 'density', 'density_atomic', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable','band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal',  'is_magnetic','ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites','theoretical']
          
X = df.loc[1:, features].values
Y = df.loc[1:, "Shape"].values


In [13]:
print(X)

[['5' '56.3792677148114' '8.311042316450266' ... '0' '0' '1']
 ['20' '277.16717209981965' '6.296939520696896' ... '2' '1' '1']
 ['20' '276.7660678753778' '6.306065383062162' ... '2' '1' '0']
 ...
 ['10' '129.16347550154512' '6.02477887435746' ... '0' '0' '1']
 ['5' '63.814802922926106' '3.54957837564461' ... '1' '1' '1']
 ['5' '51.77056123443007' '8.195022676786662' ... '0' '0' '1']]


In [14]:
for i in X[0]:
    print(i, type(i))

5 <class 'str'>
56.3792677148114 <class 'str'>
8.311042316450266 <class 'str'>
11.27585354296228 <class 'str'>
-6.239539886 <class 'str'>
-6.651739886 <class 'str'>
-2.1920820792500004 <class 'str'>
0.2562708705909085 <class 'str'>
0 <class 'str'>
0.0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
6.45797942 <class 'str'>
0 <class 'str'>
1 <class 'str'>
0 <class 'str'>
1 <class 'str'>
0.0325802 <class 'str'>
0.0005778755439819389 <class 'str'>
0.0325802 <class 'str'>
0 <class 'str'>
0 <class 'str'>
1 <class 'str'>


In [15]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split (X, Y, test_size=0.2, random_state=42)

In [16]:
oversample = SMOTE()#oversampling of data
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

In [17]:
def CRandCM(method,X_test,Y_test):#a function to print classification report and confusion matrix
    Y_pred=method.predict(X_test)
    cr = report(Y_test,  Y_pred)
    cm = confusion_matrix(Y_test, Y_pred)
    print(cr)
    print(cm)

In [18]:
#SVM_linear = svm.SVC(kernel='linear',  C=0.1,random_state=31).fit(X_train, Y_train)
#print("Accuracy:",SVM_linear.score(X_test, Y_test))
#CRandCM(SVM_linear,X_test,Y_test)


In [21]:
SVM_rbf = svm.SVC(kernel='rbf', C=1,gamma=1, verbose=True).fit(X_train, Y_train)
print("Accuracy:",SVM_rbf.score(X_test, Y_test))
CRandCM(SVM_rbf,X_test,Y_test)

[LibSVM]Accuracy: 0.9003998545983278
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      1311
           1       0.99      0.81      0.89       508
           3       0.98      0.85      0.91       191
           4       1.00      0.84      0.91       284
           5       0.99      0.79      0.88       393
           6       1.00      0.67      0.80        64

    accuracy                           0.90      2751
   macro avg       0.96      0.83      0.88      2751
weighted avg       0.91      0.90      0.90      2751

[[1308    1    2    0    0    0]
 [  92  413    0    0    3    0]
 [  27    0  163    1    0    0]
 [  42    2    0  239    1    0]
 [  77    3    2    0  311    0]
 [  21    0    0    0    0   43]]


In [22]:
#without weights
SVM_poly = svm.SVC(kernel ='poly', degree = 5, C=209.5).fit(X_train, Y_train)
print("Accuracy:",SVM_poly.score(X_test, Y_test))

Accuracy: 0.5652490003635042


In [24]:
CRandCM(SVM_poly,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.59      0.95      0.73      1311
           1       0.57      0.23      0.33       508
           3       0.16      0.05      0.07       191
           4       0.78      0.30      0.44       284
           5       0.40      0.16      0.23       393
           6       0.30      0.53      0.38        64

    accuracy                           0.57      2751
   macro avg       0.47      0.37      0.36      2751
weighted avg       0.54      0.57      0.50      2751

[[1248   33    5    0    6   19]
 [ 284  116   17    4   49   38]
 [ 166    7    9    4    3    2]
 [ 139   14    4   86   32    9]
 [ 250   30   23   15   62   13]
 [  25    2    0    1    2   34]]
